In [1]:
import sys
import collections
from collections import Counter

import datetime
from datetime import datetime, timedelta, timezone
import dateutil.parser

from collections import defaultdict

import yaml

import pandas as pd
import seaborn as sns
pd.options.display.max_rows = 999

import pytz
from pytz import timezone

import pydent
from pydent import AqSession, models
from pydent.models import Sample, Item, Plan

sys.path.append("../ext-plan-pydent")
from plans import ExternalPlan

In [2]:
filename = '../ext-plan-pydent/config.yml'

with open(filename, 'r') as f:
    config = yaml.load(f)

login = config['aquarium']['production']

session = AqSession(
    login['username'],
    login['password'],
    login['url']
)

me = session.User.where({'login': login['username']})[0]
print('Logged in as %s\n' % me.name)

Logged in as Devin Strickland



/usr/local/Cellar/ipython/7.5.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  after removing the cwd from sys.path.


In [5]:
def find_in_batches(cls, conditions):
    offset = 0
    limit = 100
    all_objs = []

    while True:
        objs = cls.where(conditions, opts={"offset": offset, "limit": limit})
        if not objs: break
        all_objs += objs
        offset += limit
        print(len(all_objs))
        
    return all_objs

In [10]:
def json_update(model, **params):
    """This update method is fairly dangerous. Be careful!"""
    data = {"model": {"model": model.__class__.__name__}}
    data.update(model.dump(**params))
    return model.session.utils.aqhttp.post('json/save', json_data=data)

In [4]:
ot = session.ObjectType.find_by_name("50 mL Agarose Gel in Gel Box")
print(ot)

{'cleanup': 'No cleanup information',
 'columns': 6,
 'cost': 0.01,
 'created_at': '2018-05-22T15:41:23.000-07:00',
 'data': 'No data',
 'description': 'Used to run gels',
 'handler': 'collection',
 'id': 785,
 'image': '',
 'max': 100,
 'min': 0,
 'name': '50 mL Agarose Gel in Gel Box',
 'prefix': '',
 'release_description': '',
 'release_method': 'return',
 'rid': 7,
 'rows': 2,
 'safety': 'No safety information',
 'sample_type_id': None,
 'unit': 'box',
 'updated_at': '2018-05-22T15:49:37.000-07:00',
 'vendor': 'No vendor information'}


In [7]:
conditions = {"object_type_id": ot.id}
items = find_in_batches(session.Item, conditions)
print(len(items))

100
195
195


In [8]:
locations = list(set([i.location for i in items]))
print(locations)

['Bench', 'Gel Room', 'deleted']


In [15]:
undeleted = [i for i in items if not i.location == 'deleted']
print(len(undeleted))

0


In [13]:
print(str(list(i.quantity for i in undeleted)))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [14]:
for item in undeleted:
    item.location = 'deleted'
    json_update(item)